# Data Scraping

In [3]:
# import necessary libraries
import requests
from bs4 import BeautifulSoup as bs
from time import *
from random import randint


url = "https://en.wikipedia.org/wiki/List_of_chemistry_journals"

def get_list_journals_page(url):
    
    """
    send request to a server to get a page of all journals listed
    """
    
    # specify the parameters of the browser settings
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) \
    Chrome/83.0.4103.61 Safari/537.36'}
    page = requests.get(url, headers = header)
    bsobj = bs(page.content,'lxml')
    return [page.status_code, bsobj]

In [21]:
# getting status code and the soup object
status_code = get_list_journals_page(url)[0]
print('Status code', status_code)

bsobj = get_list_journals_page(url)[1]

Status code 200


In [9]:
def links_to_journal(bsobj):
    
    """
    search soup object (bsobj) to find links to a specific journal
    """
 
    links = []
    for li in bsobj.findAll('li'):
        if not li.i:
            continue
        links.append(li.i.a['href'])
    return links

In [22]:
links = links_to_journal(bsobj)

In [15]:
def full_journal_path(links):
    
    """
    compile the full path for a journal,
    that consis from the base_path and the individual links-pointers
    """

    base_path = "https://en.wikipedia.org/"

    full_links = []
    for link in links:
        full = base_path + link
        full_links.append(full)
    return full_links

In [23]:
full_links = full_journal_path(links)

In [17]:
# sample scraping, for full scraping remove list slice
full_links = full_links[:5]

# dict to store key and values
movie_info = {}

# list to append dict
get_info = []

#tracker of pages
n = 0

# traker of links
tracker = list()


def dataframe_creation(movie_info, get_info,tracker, n, full_links):
    
    """
    iterate across full links and add it to a tracker 
    to avoid scraping the same link again
    """
    
    for link in full_links:
        if link in tracker:
            continue
        tracker.append(link)

        """
        to avoid being blocked due to sending excessive server requests
        """
        x = randint(10,17)
        sleep(x)
        print('time is', x, 'scraping page', n)

        r = requests.get(link)
        soup = bs(r.content)

        """
        for individual links find table to scrape in soup object
        and create a dictionary where keys would be the th and values are td
        """
        
        info_box = soup.find(class_="infobox hproduct")
        try:
            info_rows = info_box.find_all("tr")
        except:
            continue

        for index, row in enumerate(info_rows):
            if not row.find('th'):
                continue
            content_key = row.find("th").get_text(" ", strip=True)
            try:
                content_value = row.find("td").get_text()
            except:
                pass
            movie_info[content_key] = content_value

        """
        as title not in that class ("infobox hproduct") search the entire soup object 
        to find journal title and its description
        """
        
        for k in info_box.find_all('span', {'class':"fn"}):
            res = k.text
        movie_info['title'] = res

        try:
            p = soup.find(class_="mw-parser-output")
            parag = p.p.get_text()
            movie_info['description'] = parag
        except:
            pass
        get_info.append(movie_info)
        movie_info = {}
        n+=1

    df = pd.DataFrame(get_info)

    return df

In [18]:
df = dataframe_creation(movie_info, get_info,tracker, n, full_links)

time is 15 scraping page 0
time is 12 scraping page 1
time is 16 scraping page 2
time is 16 scraping page 3
time is 13 scraping page 4


<IPython.core.display.Javascript object>

In [20]:
# get the datasset head
df.head()

,Discipline,Language,Edited by,Publication details,History,Publisher,Frequency,Impact factor,Standard abbreviations ISO 4 ( alt ) · Bluebook ( alt1 · alt2 ) NLM ( alt ) · MathSciNet ( alt ),ISO 4,...,CODEN,ISSN,LCCN,OCLC no.,Links,title,description,Standard abbreviations ISO 4 ( alt ) · Bluebook ( alt1 · alt2 ) NLM · MathSciNet,Former name(s),Open access
0,"Chemistry, biochemistry",English,Cynthia J. Burrows,Cynthia J. Burrows,1968–present,American Chemical Society (United States),Monthly,22.384 (2020),22.384 (2020),Acc. Chem. Res.,...,ACHRE4,0001-4842 (print)1520-4898 (web),70008300,475052789,475052789,Accounts of Chemical Research,Accounts of Chemical Research is a monthly pee...,NaN,NaN,NaN
1,Chemistry,"English, German, French",NaN,"English, German, French",1947-1999,"Danish Chemical Society, Finnish Chemical Soci...",NaN,NaN,NaN,Acta Chem. Scand.,...,NaN,NaN,NaN,NaN,Acta Chem. Scand.,Acta Chemica Scandinavica,Acta Chemica Scandinavica was a peer-reviewed ...,"Danish Chemical Society, Finnish Chemical Soci...",NaN,NaN
2,Chemistry,"English, Slovene",Aleksander Pavko,Aleksander Pavko,1954–present,Slovenian Chemical Society (Slovenia),Quarterly,1.104 (2017),1.104 (2017),Acta Chim. Slov.,...,ACSLE7,1318-0207 (print)1580-3155 (web),94646636,29881395,29881395,Acta Chimica Slovenica,Acta Chimica Slovenica is a quarterly scientif...,NaN,Vestnik Slovenskega kemijskega društva,NaN
3,Materials Science,English,Joern Ritterbusch,Joern Ritterbusch,1985–present,Wiley-VCH (Germany),weekly,18.808 (2020),18.808 (2020),Adv. Funct. Mater.,...,AFMDC6,1616-301X (print)1616-3028 (web),2001205770,NaN,2001205770,Advanced Functional Materials,Advanced Functional Materials is a peer-review...,NaN,Advanced Materials for Optics and Electronics ...,some
4,Chemistry,English,NaN,English,1968–present,Sigma-Aldrich (USA),NaN,17.083[1] (2014),17.083[1] (2014),Aldrichimica Acta,...,ALACBI,0002-5100,NaN,NaN,0002-5100,Aldrichimica Acta,Aldrichimica Acta is a scientific journal publ...,NaN,NaN,NaN


In [108]:
# save dataset for future use
df.to_csv('chem_journ.csv')

# Data Cleaning

In [24]:
# import libraries
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# import dataframe
df = pd.read_csv(r'C:\Users\valer\OneDrive\Desktop\Python\chem_journ.csv')

In [25]:
df.head()

,Unnamed: 0,Discipline,Language,Edited by,Publication details,History,Publisher,Frequency,Impact factor,Standard abbreviations ISO 4 ( alt ) · Bluebook ( alt1 · alt2 ) NLM ( alt ) · MathSciNet ( alt ),ISO 4,Indexing CODEN · JSTOR ( alt ) · LCCN ( alt ) MIAR · NLM ( alt ) · Scopus,CODEN,ISSN,LCCN,OCLC no.,Links,title,description,Standard abbreviations ISO 4 ( alt ) · Bluebook ( alt1 · alt2 ) NLM · MathSciNet,Former name(s),Open access,NLM,License,Editor,Categories,Year founded,Country,Website,OCLC,Indexing CODEN · JSTOR · LCCN MIAR · NLM · Scopus
0,0,"Chemistry, biochemistry",English,Cynthia J. Burrows,Cynthia J. Burrows,1968–present,American Chemical Society (United States),Monthly,22.384 (2020),22.384 (2020),Acc. Chem. Res.,Acc. Chem. Res.,ACHRE4,0001-4842 (print)1520-4898 (web),70008300,475052789.0,475052789,Accounts of Chemical Research,Accounts of Chemical Research is a monthly pee...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Chemistry,"English, German, French",NaN,"English, German, French",1947-1999,"Danish Chemical Society, Finnish Chemical Soci...",NaN,NaN,NaN,Acta Chem. Scand.,NaN,NaN,NaN,NaN,NaN,Acta Chem. Scand.,Acta Chemica Scandinavica,Acta Chemica Scandinavica was a peer-reviewed ...,"Danish Chemical Society, Finnish Chemical Soci...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Chemistry,"English, Slovene",Aleksander Pavko,Aleksander Pavko,1954–present,Slovenian Chemical Society (Slovenia),Quarterly,1.104 (2017),1.104 (2017),Acta Chim. Slov.,Acta Chim. Slov.,ACSLE7,1318-0207 (print)1580-3155 (web),94646636,29881395.0,29881395,Acta Chimica Slovenica,Acta Chimica Slovenica is a quarterly scientif...,NaN,Vestnik Slovenskega kemijskega društva,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Materials Science,English,Joern Ritterbusch,Joern Ritterbusch,1985–present,Wiley-VCH (Germany),weekly,18.808 (2020),18.808 (2020),Adv. Funct. Mater.,Adv. Funct. Mater.,AFMDC6,1616-301X (print)1616-3028 (web),2001205770,NaN,2001205770,Advanced Functional Materials,Advanced Functional Materials is a peer-review...,NaN,Advanced Materials for Optics and Electronics ...,some,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Chemistry,English,NaN,English,1968–present,Sigma-Aldrich (USA),NaN,17.083[1] (2014),17.083[1] (2014),Aldrichimica Acta,Aldrichimica Acta,ALACBI,0002-5100,NaN,NaN,0002-5100,Aldrichimica Acta,Aldrichimica Acta is a scientific journal publ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# get only those columns that are important for a project
df = df[['Discipline', 'Language', 'Edited by', 'History',
    'Frequency', 'Impact factor', 'description',
    'title']]

In [27]:
def data_preprocessing(df):
    
    """
    extract the imact factor number from a given column, convert its value to float and drop the initial column 
    """
    df['Impact_factor'] = df['Impact factor'].str.replace('(','').str.replace(')','').str.replace(r'\d{4,7}','').str.replace('\[\d+]', '')
    df['Impact_factor'] = pd.to_numeric(df['Impact_factor'], errors = 'coerce')
    df['Impact_factor'] = df['Impact_factor'].astype(float)
    df.drop('Impact factor', axis = 1, inplace = True)
    
    df['description'] = df['description'].str.lower()
    
    
    return df

In [28]:
df = data_preprocessing(df)

In [29]:
def get_index(key_words):
    
    """
    getting index of the journal if the key words match the journal description info
    """
    res = key_words.lower().split()
    lst_index = []
    
    for word in res:
        for row in range(df.shape[0]):
            if df['description'].str.contains(word)[row]:
                lst_index.append(row)
    return  lst_index


def dic_journals(lst_index):
    
    """
    get the title and the impact factor info based om the index match from get_index function
    """
    lst_journals = []
    lst_factor = []
    dic = {}

    for row in lst_index:

        journal_name = df['title'][row]
        impact_factor = df['Impact_factor'][row]
        lst_factor.append(impact_factor)
        lst_journals.append(journal_name)

        dic = dict(zip(lst_journals, lst_factor))
    return dic

def create_dataframe(dic):
    
    """
    create a dataset based on the dict and sort values by impact factor
    """   
    df_new = pd.DataFrame(dic.values(), dic.keys()).reset_index()
    df_new.columns = ['title', 'Impact_factor']
    df_new = df_new.sort_values(by = 'Impact_factor', ascending = False)
    return df_new

please insert the key words of your project, like Analytical Chemistry, Spectroscopy and so on: chemistry


# Cleaned Dataset

In [30]:
key_words = input('please insert the key words of your project, like Analytical Chemistry, Spectroscopy and so on: ')

lst_index = get_index(key_words)
dic = dic_journals(lst_index)
df_new = create_dataframe(dic)
df_new

please insert the key words of your project, like Analytical Chemistry, Spectroscopy and so on: analytical nanomaterials


,title,Impact_factor
4,Annual Review of Analytical Chemistry,10.745
13,Trends in Analytical Chemistry,9.801
3,Analytical Chemistry,6.785
11,Microchimica Acta,6.232
1,Analytica Chimica Acta,5.977
12,Talanta,5.339
5,Bulletin of the Chemical Society of Japan,4.488
6,Geostandards and Geoanalytical Research,4.256
8,Journal of Chromatography A,4.169
0,Analyst,3.864
